# 2021/1/11（鈴木・第４回）再帰関数と汎用的な全探索(1)

## 再帰関数とは

再帰関数とは、自分自身を呼び出す関数のこと。
なにが嬉しいのか？

- `itertools`を使っても書けないような複雑な繰り返しを記述できる
- **分割統治法:** より小さな問題を解く。その結果を使ってもとの問題を解く。
    - たとえば漸化式はその例。
    - 類似の内容だけど、「長さ $n$ の列に関する問題がある。最後の項の値で場合分けする。それぞれについて長さ $n-1$ の問題を解けばよい」とか。
    - 問題を半分半分に分けて解いた結果を統合することで、計算の効率が上がるときがある（マージソート、クイックソート、高速フーリエ変換）

- 木構造など、そもそも再帰的に定義される対象を扱う。
    - 根付き木は「根」＋「部分木」＋「部分木」＋ ... + 「部分木」の構造を持つ

##具体例: 和の計算

$f(n) = 1 + \cdots + n$ の値を計算する。
以下の漸化式をもとに再帰関数を定義して計算する。

- **ベースケース:** $f(0) = 0$
- **漸化式:** $f(n) = f(n-1) + n$

これらの性質を再帰関数に落とし込むと次のように実装できる。



In [ ]:
def f(n):
    if n == 0: # ベースケース
        return 0
    # それ以外の場合
    return f(n-1) + n

print(f(5))

15


上のコード内で、「初期値」と「漸化式」が両方記述されていて、「漸化式」のほうでは自分自身を呼び出していることに注意してほしい。

再帰関数の「初期値」にあたるものをベースケースと呼ぶ。


## お気持ち語り

漸化式は、$f(0), f(1), f(2), \dots$ と順に値を求めるイメージを持っている人が多いと思う。
再帰はむしろゴールから逆向きにたどっていって、スタート地点（ベースケース）にたどりついたら、その結果をもって引き返す感じ。

- いま、$f(3)$ を知りたい。$f(3) = f(2) + 3$ だ。ところで $f(2)$ って何？
- よって $f(2)$ を知りたい。$f(2) = f(1) + 2$ だ。ところで $f(1)$ って何？
- よって $f(1)$ を知りたい。$f(1) = f(0) + 1$ だ。ところで $f(0)$ って何？
- $f(0)$ は、知ってる！ $f(0) = 0$ だ！（ベースケース）
- ってことは、 $f(1) = 1$ だ！
- ってことは、 $f(2) = 3$ だ！
- ってことは、 $f(3) = 6$ だ！めでたしめでたし。

## 無限ループにご注意を

再帰関数の実装を失敗すると無限ループになってしまうことが多い。ありがちなミスを挙げよう。

- ベースケースを実装し忘れる
- ベースケースにたどり着かない入力がある
    - 上の例だと、$f(-1)$ を計算しようとすると無限ループになる
- 問題サイズが小さくなってない

いずれにせよ、数秒待ってもプログラムが終わらない場合は停止ボタンを押したあと、上のようなミスをしていないか考え直してみよう。

### 具体例：フィボナッチ数とメモ化再帰

フィボナッチ数列 $1,2,3,5,8, \dots$ は漸化式
$$
f(1) = 1,\quad f(2) = 2,\quad f(n) = f(n-1) +f(n-2)
$$
をみたす。これを再帰関数に落とし込むと次のコードになる。


In [ ]:
def fib(n):
    if n == 1:
        return 1
    if n == 2:
        return 2
    return fib(n-1) + fib(n-2)

print(fib(10))

89


しかしこのコードは非効率である。たかが `fib(36)` の計算に５秒ほどかかってしまうことを確認してみよう。

In [ ]:
fib(36)

24157817

なぜこんなにも効率が悪くなってしまうのだろうか。
フィボナッチ数の計算でおこる関数の呼び出しを書いてみよう（ここで黒板に書く）。
$n$ について指数的に呼び出し回数が増えることがわかる。

黒板を見ればわかるように、この再帰では同じ計算を何度も何度も繰り返して、とても効率が悪い。
代わりに一度計算された値を覚えるようにすれば、効率的なプログラムになる。
これを**メモ化再帰**という。

「一度計算された値を覚える」（メモする、キャッシュする）ためには、
- 事前に入力の値の範囲がわかっているときはリストに確保
- 辞書を使って計算値を記憶する
- python の関数 `lru_cache` を使う

などの方法がある。ここでは `lru_cache` を使う例を紹介する。
@ の行はデコレータという機能を使っている（そういう文法だとまる覚えしたほうがよい）。

#### 注意
もちろん、いつでもメモ化が効率的とは限らない。メモ化が効率的になるのは何回も同じ計算をするときである。再帰呼び出しの過程で毎回異なる計算をする場合は効率は全くよくならない。（むしろメモする分、時間もメモリも無駄にすることになる）


In [3]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n == 1:
        return 1
    if n == 2:
        return 2
    return fib(n-1) + fib(n-2)

print(fib(100))

573147844013817084101


## スタックオーバーフロー

ただし上のコードで `fib(1000)`を計算しようとするとエラーが出てしまう。
再帰呼び出しをするとき、コンピュータの内部では「戻る場所」「関数内部で定義した変数の値」などのデータを（スタック領域に）覚えている。
再帰呼び出しを何重（何百重）にも繰り返すと、
「覚える空きスペースがない！」となってしまい、エラーとなる。
これをスタックオーバーフローという。


## 具体例：ユークリッドの互除法

ここまでは、再帰関数とはいっても for 文で求まるものばかりで、あまり嬉しさがわからなかったかもしれない。しかし再帰は本当に強力である。もっと例を見てみよう。


$a,b$ の最大公約数 $\gcd(a,b)$ を効率的に求める方法として、ユークリッドの互除法がある。
ユークリッドの互除法では、以下の性質を使って最大公約数を求める。

- $\gcd(a,0) = a$（$0$ はあらゆる数の倍数である）
- $b \neq 0$ のとき、あまり付きの割り算 $a = qb + r$ を考えると、$\gcd(a,b) = \gcd(b,r)$ である

さあ、ユークリッドの互除法を実装しよう。
$\gcd(470434748396536, 76206123870137184) = 2181038552$ を確認しよう。


（ちなみに、python の math ライブラリには `gcd` という最大公約数を求めてくれる関数があります）




In [ ]:
def gcd(a,b):
    if b == 0:
        return a
    return gcd(b,a%b)

print(gcd(470434748396536, 76206123870137184))

2181038552


## 具体例：ハノイの塔

- $3$ 本の杭 ($0,1,2$ と番号付けする) と、中央に穴の開いた大きさの異なる $N$ 枚の円盤がある。
- 最初はすべての円盤が、左端の杭に小さいものが上になるように順に積み重ねられている。
- 円盤を一回に一枚ずつどれかの杭に移動させることができるが、小さな円盤の上に大きな円盤を乗せることはできない。

この条件のもとで、すべての円盤を右端の杭に移動させたい。その最短手順を求めよ。（図を黒板に書く。）

**解答例**

問題に補助変数を導入して、杭 $a$ から杭 $b$ に $N$ 枚の円盤を移動させる、とする。以下のようにすると、 $N-1$ のときの解答を使って $N$ のときの解答が作れる
- 上 $N-1$ 枚を $a$ から $c$ に動かす。ここで $c$ は $a$ とも $b$ とも異なる（$c=3-a-b$ として簡単に計算できる）
- 一番大きい円盤を $a$ から $b$ に動かす。
- $N-1$ 枚を $c$ から $b$ に動かす。

ベースケースは $N=0$ の場合で、「何もしない」が解である。

以上を再帰関数に落とし込むと以下のようになる。


In [ ]:
def hanoi(n,a,b):
    if n == 0:
        return
    
    c = 3-a-b # スタートでもゴールでもない杭の位置
    hanoi(n-1,a,c)
    print(f"from {a} to {b}") # 「f文字列」 という記法で、文字列の一部を変数に依存させたいときの書き方。
    hanoi(n-1,c,b)

hanoi(3,0,2)



from 0 to 2
from 0 to 1
from 2 to 1
from 0 to 2
from 1 to 0
from 1 to 2
from 0 to 2


## 具体例：フラクタル図形

ここでは、レベル $L$ のコッホ曲線を書く疑似アルゴリズムを考えよう。コッホ曲線の説明は黒板に書く（以下のリンク）
https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%83%E3%83%9B%E6%9B%B2%E7%B7%9A

レベル 0 のコッホ曲線は直線、レベル 1 は ＿／＼＿ のような形になる。
コッホ曲線は、以下のような疑似コードで書くことができる（黒板で実例を見せる）
```
def Koch(L):
    if L == 0:
        長さ 1 直進
    Koch(L-1)
    左に60度回転
    Koch(L-1)
    右に120度回転
    Koch(L-1)
    左に60度回転
    Koch(L-1)
```

## 具体例：木構造（ディレクトリ構造を例に）

**問題**
とあるフォルダの中のファイル名をすべて知りたい。
ただし、フォルダの中にフォルダが入っていることもあるし、何重の入れ子構造になっているか知らない。
どうすればよいか？
ただし、以下の関数が使えるとする

- `os.listdir(path)`: path というディレクトリの中のファイル名およびディレクトリ名のリストを出力
- `os.path.isdir(path)`: path がディレクトリなら `True`, そうでないなら `False`

**説明**

（根付き）木構造は以下のように定義される（そもそも定義が再帰的であることに注意！）

- １点は木である
- 根に木をいくつかくっつけたものは木である

ディレクトリ構造は木構造なので、再帰関数を使えばディレクトリを再帰的にたどることができる。

${}$
${}$


以下の例のように、ファイル名やディレクトリ名の一覧を取得できる。


In [ ]:
import os
files = os.listdir('./') # ディレクトリの中のファイル名やディレクトリ名の一覧
print(files)


['.config', 'sample_data']


すると、フォルダを開く関数を再帰的に実行すれば、すべてのフォルダについて、中身の一覧が取得できる。

- 「すべてのディレクトリの中身を見る」とは、
    - 今いるところにあるディレクトリすべてについて、
    - 「すべてのディレクトリの中身を見る」こと


In [ ]:
import os
def open_all_directories(dir):
    names = os.listdir(dir)            # ディレクトリの中身のリスト
    print(dir)
    print(names)                   # ディレクトリの中身のリストをプリント
    for file in names:       # すべてのファイル or ディレクトリについて、
        path = os.path.join(dir, file)
        if os.path.isdir(path):        # もしディレクトリなら
            open_all_directories(path) # 再帰的に実行

open_all_directories('./')


./
['.config', 'sample_data']
./.config
['.last_update_check.json', 'logs', 'config_sentinel', '.last_survey_prompt.yaml', 'active_config', 'gce', 'configurations', '.last_opt_in_prompt.yaml']
./.config/logs
['2022.01.07']
./.config/logs/2022.01.07
['14.32.37.114755.log', '14.32.57.118850.log', '14.33.22.232212.log', '14.33.42.565580.log', '14.33.15.504888.log', '14.33.41.864886.log']
./.config/configurations
['config_default']
./sample_data
['README.md', 'anscombe.json', 'california_housing_train.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_test.csv']


## 練習問題

### 練習問題１: 階乗

$f(n) = n!$ の値を、以下の漸化式を再帰関数に落とし込むことで求めよう。

- $f(1) = 1$
- $f(n) = n \cdot f(n-1)$

とくに $f(6) = 720$ となることを確認しよう。



In [ ]:
## ここに書いてみよう






**解答例**

In [ ]:
def f(n):
    if n == 1:
        return 1
    else:
        return n * f(n-1)

print(f(6))


720


### 練習問題２（桁和）

$N$ の $10$ 進表記での各桁の和を $f(N)$ とする。
たとえば $f(314) = 3+1+4 = 8$ である。
$f$ を以下に基づく再帰関数で実装せよ。

- $n = 10q+r$ のとき、 $f(n) = f(q) + r$
- $f(0) = 0$

$f(1234567890) = 45$ を確認すること。


In [ ]:
### ここに書いてみよう













**解答例**

In [ ]:
def f(n):
    if n == 0:
        return 0
    return f(n//10) + n%10

print(f(1234567890))

45


### 練習問題３（二項係数を漸化式から計算する）

二項係数 $\mathrm{binom}(n,k) = \frac{n!}{k!(n-k)!}$ を、以下の方針で求めてみよう。

- 漸化式 $\mathrm{binom}(n,k) = \mathrm{binom}(n-1,k-1) + \mathrm{binom}(n-1,k)$ を使ってメモ化再帰する
- ベースケースは、$\mathrm{binom}(0,0)$, $\mathrm{binom}(n,-1)$, $\mathrm{binom}(n,n+1)$ を適切に設定する（つまり、パスカルの三角形の頂点と外側）

そして $\mathrm{binom}(30,15) = 155117520$ であることを確かめよ。（メモ化を忘れずに！）

In [ ]:
# ここに書いてみよう













**解答例**

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def binom(n,k):
    if n == 0 and k == 0:
        return 1
    elif k < 0 or k > n:
        return 0
    else:
        return binom(n-1,k-1) + binom(n-1,k)

print(binom(30,15))

# 以下は本題とは関係ないが、階乗で計算する方法
# ver 3.8 以降では math.comb(n,k) というそのまんまの関数があるが、colab にある python は 3.7 なので使えない
from math import factorial
def f(n,k):
    return factorial(n)//factorial(k)//factorial(n-k)
print(f(30,15))




155117520
155117520


## 練習問題：グレイコード（難しいかも）

$N$ ビットのグレイコードとは、以下の条件をみたすビット列の列である。
- 各要素は $0,1$ からなる長さ $N$ の列（ビット列）
- グレイコードの中にそのようなビット列が１回ずつあらわれる（つまりグレイコードは $2^N$ 個のビット列からなる）
- どの隣同士のビット列をとってきても、その 2 つのビット列で異なるビットはちょうど１つ

たとえば、以下の列は $3$ ビットのグレイコードである。
$$
000,001,011,010,110,111,101,100
$$

- $N$ ビットのグレイコード（をあらわす、文字列を要素とするリスト）を出力する関数 $f(N)$ を実装せよ
- $f(3)$ や $f(4)$ を出力するなどして正しさを確かめよ。


In [ ]:
# ここに書いてみよう














**解答例**

$N-1$ のときのグレイコード r があったとき、
- r の各ビット列の先頭に $0$ をつけたもの
- r を逆順にして、各ビット列の先頭に $1$ をつけたもの

をそのままつなげれば $N$ のときのグレイコードになる。

ベースケースは $N=0$ のときで、空文字列（長さ 0 のビット列）からなる長さ 1 の列がグレイコードとなる。

In [ ]:
def gray(n):
    if n == 0:
        return [""]
    R = gray(n-1)
    R0 = ["0" + i for i in R]        # 内包表記を利用すると、「すべての文字列の頭に0をつける」がカンタンにできる
    R1 = ["1" + i for i in R[::-1]]  # R[::-1] で、逆順にした文字列をあらわす
    return R0 + R1

print(gray(3))


## 練習問題：マージソート（難しいかも）

**ソート**とは、リストの要素を小さい順に並べることである。たとえば、
$[3,1,4,1,5,9,2,6,5,3,5]$ をソートすると $[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]$ となる。

python には組み込み関数 `a.sort()` や `sorted(a)` があるが、
ここでは「マージソート」というアルゴリズムを自前で実装してみよう。

マージソートのアイデアは、
- 列を前半分と後ろ半分の２つに分ける
    - リスト $A$ の長さ `len(A)` が $n$ のとき、$A$ の前半分は `A[:n//2]`, 後ろ半分は `A[n//2:]` と書ける
- それぞれをソートする（サイズが半分の問題を２回解く）
- ソート済み列２つをくっつけて（マージして）１つのソート済み列を作る
    - （実は、ここでソート済み列のマージは効率的にできるという事実を使っている）

というものである。ベースケースは要素が 1 つの列で、すでにソート済みである。

リスト $A$ を入力とし、$A$ をソートした列を返す関数 `merge_sort(A)` を実装せよ。
ここで、 ソート済み列 $A,B$ をマージして１つのソート済み列を返す関数 `merge(A,B)` は実装しておいた。部品として使ってもよい。



In [ ]:
# ご自由にお使いください
def merge(A,B):
    C = A + B[::-1]
    L,R = 0, len(C)-1
    res = []
    while R-L >= 0:
        if C[L] <= C[R]:
            res.append(C[L])
            L += 1
        else:
            res.append(C[R])
            R -= 1
    return res

## ここに解答を書いてください












**解答例**

In [ ]:
def merge_sort(A):
    n = len(A)
    if n == 1:
        return A
    A0 = merge_sort(A[:n//2])
    A1 = merge_sort(A[n//2:])
    return merge(A0,A1)



x = merge_sort([3,1,4,1,5,9,2,6,5,3,5])
print(x)


z = list(range(10**4)[::-1]) # 逆順
print(merge_sort(z) == sorted(z)) # 本当にソートされているか確認


[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]
True
